# Script for visualization of fineSTRUCTURE results in R

## Before you begin 

In [ ]:
#install.packages("tidyverse")
install.packages("ape")
install.packages("XML")
install.packages("reshape2")
install.packages("cowplot")

## Visualisation of distances using a function from fineSTRUCTURE: coancestry and IBS

In [ ]:
library(ape)
library(XML)
library(tidyverse)

setwd("../..")

source("bin/fs_4.1.1/FinestructureLibrary.R")

# Tree and order of samples

treexml <- xmlTreeParse("results/4.finestructure/saimaa_20ctgs_linked_tree.xml")  ## read the tree as xml format
ttree <- extractTree(treexml) ## extract the tree into ape's phylo format
ttree$node.label <- c(1:97)

#At this point, you may want to rotate some parts of the tree to make a nicer plot by using function rotate(ttree, number of parent node)

tdend <- myapetodend(ttree,factor=1) # convert to dendrogram format

fullorder <- labels(tdend) # the order according to the tree
fullorder <- gsub("s","",fullorder)

###############################

#Coancestry data and plotting using function from fineSTRUCTURE

data_df <- read.table("results/4.finestructure/saimaa_20ctgs_linked.chunkcounts.out", row.names=1,header=T,skip=1)

dataraw <- as.matrix(data_df) # read in the pairwise coincidence 
colnames(dataraw) <- gsub("s","",colnames(dataraw))
rownames(dataraw) <- gsub("s","",rownames(dataraw))

datamatrix <- dataraw[fullorder,fullorder] # reorder the data matrix

write.table(datamatrix, "results/4.finestructure/datamatrix.txt", row.names=T,quote=F, col.names=T)
write.table(fullorder, "results/4.finestructure/fullorder.txt", row.names=F,quote=F, col.names=F)

fsmax <- 395 # cap the heatmap, threshold set on the highest 1.5% of values
fsmat <- datamatrix 
fsmat[fsmat>fsmax] <- fsmax

pca_cols <- read.table("results/3.smartpca/colors.txt", sep = "\t", header=F)

k <- match(fullorder,pca_cols$V1)
lcol <- pca_cols$V1[k]

colfunc<-colorRampPalette(c("papayawhip","#edc861","goldenrod", "orangered4", "purple4", "royalblue"))
some.colorsEnd<-(colfunc(40))
some.colorsEnd[40]="gray20"

png("results/4.finestructure/coancestry.png", height = 900, width = 900)
plotFinestructure(fsmat,fullorder,cols=some.colorsEnd,cex.axis=0.6,
                  edgePar=list(p.lwd=0,t.srt=90,t.off=-0.1,t.cex=0.8),text.col=lcol)
dev.off()

#############################
#IBS data and plotting using function from fineSTRUCTURE

ibs <- read.table("processed_data/4.finestructure/IBS/plink.mibs", sep = "\t")
inds <- read.table("data/saimaa_IDs.txt")

ibsmatrix <- as.matrix(ibs)
colnames(ibsmatrix) <- inds$V1
rownames(ibsmatrix) <- inds$V1

ibsmatrix <- ibsmatrix[fullorder,fullorder] # Order of matrix changed to match that of the finestructure matrix
diag(ibsmatrix) <- NA

ibsmax <- 0.87 # cap the heatmap, threshold set on the highest 1.5% of values
ibsmat <- ibsmatrix
ibsmat[ibsmat>ibsmax] <- ibsmax # 

png(file="processed_data/4.finestructure/IBS/identitybystate.png",height=900,width=900)
plotFinestructure(ibsmat,fullorder,cols=some.colorsEnd,cex.axis=0.6,
                  edgePar=list(p.lwd=0,t.srt=90,t.off=-0.1,t.cex=0.8),text.col=lcol)
dev.off()


## Plot the metrics using ggplot2 (included in the publication)

In [ ]:
library(reshape2)
library(cowplot)

fs_melt <- melt(fsmat)
fs_melt$Var1 <- factor(as.character(fs_melt$Var1), levels = unique(as.character(fs_melt$Var1)))
fs_melt$Var2 <- factor(as.character(fs_melt$Var2), levels = unique(as.character(fs_melt$Var2)))

ibs_melt <- melt(ibsmat)
ibs_melt$Var1 <- factor(as.character(ibs_melt$Var1), levels = unique(as.character(ibs_melt$Var1)))
ibs_melt$Var2 <- factor(as.character(ibs_melt$Var2), levels = unique(as.character(ibs_melt$Var2)))

fs_col = c("papayawhip","navajowhite1", "#edc861", "darkred", "purple4")

fstheme <- theme(axis.text.x = element_text(angle=90),
                 axis.text = element_text(colour = lcol),
                 axis.title = element_text(size = 14),
                 legend.position = "top", 
                 legend.title = element_text(size = 13),
                 legend.text = element_text(size = 14),
                 legend.margin=margin(0,50,0,0), 
                 legend.spacing.x = unit(0.5, 'cm'))

fs_plot <- ggplot(data = fs_melt, aes(x=Var2, y=Var1, fill=value)) + 
  geom_tile()+
  xlab("Seal2")+ylab("Seal1")+
  fstheme+
  guides(fill = guide_colourbar(barwidth = 18, barheight = 0.8))+
  scale_fill_gradientn(name = "Coancestry", breaks = c(0,100,200,300,400,500,600), colours = fs_col)

ibs_plot <- ggplot(data = ibs_melt, aes(x=Var2, y=Var1, fill=value)) + 
  geom_tile()+
  xlab("Seal2")+
  fstheme+
  guides(fill = guide_colourbar(barwidth = 18, barheight = 0.8))+
  scale_fill_gradientn(name = "IBS", breaks = c(0.7,0.75,0.8,0.85,0.9,0.95,1), colours = fs_col)

fs_grid <- plot_grid(fs_plot, ibs_plot+theme(plot.margin = unit(c(0.4,0.4,0.4,1.2), ncol = 2, nrow = 1, labels = c("A", "B"))

ggsave("results/4.finestructure/coancestry_grid.png", fs_grid)